<a href="https://colab.research.google.com/github/ahxlzjt/MedImagingDL/blob/CH02_-Medical-Image-Processing-Tasks%2C-Applications%2C-and-Evaluation-metrics/ch04_05_prostate_cancer_grade_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
* 이번시간에서는 가장 data의 학습에서 중요하고 기반이 되어지는 Exploratory Data Analysis(EDA)에 대해서 알아보도록 하겠습니다
* Medical image에서 가장 중요한건 image에 대한 특성을 파악하고 feature를 추출하는 기반을 찾는게 중요한데요
* 이번 시간에서는 pathology의 data와 label에 대해서 알아보면서 이를 기반으로 Patch까지 적용하여 미리 학습에 적합한 데이터를 만드는 과정까지 진행해보도록 하겠습니다.

### Prostate Cancer?
전립선에 발생하는 암.  
남성에게 가장 흔한 암중에 하나로써 천천히 자라나지만, 급격하게 빠르게 퍼질수도 있다.  
암에 대해서 score를 측정하는 것이 중요함으로 이를 gleason score라고 부른다.

### GLEASON Score?
생검(biopsy)을 통해서 암의 Grade의 정도를 판독하는 지표   
Grade가 높을수록 전이가 큰 것을 의미함.  
Gleason score은 두 점수를 합하여 2~10까지의 점수로 판독함  


### GLEASON score to ISUP grade
International Society of Urological Pathology (ISUP) 가이드라인을 통해서 최종 Grade를 정의함.


* Gleason score 6 = ISUP grade 1
* Gleason score 7 (3 + 4) = ISUP grade 2
* Gleason score 7 (4 + 3) = ISUP grade 3
* Gleason score 8 = ISUP grade 4
* Gleason score 9-10 = ISUP grade 5

### How to get GLEASON score ?
1. 생검을 하고 난후 조직에 haematoxylin & eosin (H&E)로 염색을 함.
2. WSI에서 흰색 구멍이 또는 속이 빈 구조의 경우 GLEASON score로 판독함.

# 1. Load image
* 이미지를 읽기 위해서 일단 `openslide`를 통한 분석을 진행해도록 한다.

In [ ]:
!pip install openslide-python

In [ ]:
## Path관련 library
from pathlib import Path

## WSI관련 library
import openslide
import cv2

## etc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
class cfg:
    # Location of the training images
    BASE_PATH = '../input/prostate-cancer-grade-assessment'
    # image and mask directories
    data_dir = f'{BASE_PATH}/train_images'
    mask_dir = f'{BASE_PATH}/train_label_masks'
    train_zip_path = 'train.zip'
    mask_zip_path = 'masks.zip'
    sz = 256
    N = 16



In [ ]:
# Location of training labels
train = pd.read_csv(f'{cfg.BASE_PATH}/train.csv')
train.head()

In [ ]:
train['img_path'] = train['image_id'].apply(lambda x : Path(cfg.data_dir)/ (x+'.tiff'))
train['mask_path'] = train['image_id'].apply(lambda x : Path(cfg.mask_dir)/ (x+'_mask.tiff'))
train.head()

In [ ]:
# check NAN value
train.isna().sum()

# 2.EDA
* 기본적인 data의 distribution의 정보를 파악하는 것이 몹시 중요!   
* 데이터의 따른 학습방법을 설정하고 시험을 진행할수 있음  
* dataframe에 대해서 본격적인 분석을 진행을 해보자.

In [ ]:
import seaborn as sns # data에 대한 distribution등 다양한 feature를 분석하기 위해 사용함.

In [ ]:
train.head()

[링크 텍스트](https:// [링크 텍스트](https://))seaborn의 color나 plot을 이쁘게 해주기 위해서 setting건들여보자

In [ ]:
# Save a palette to a variable:
palette = sns.color_palette("Set2")

# Use palplot and pass in the variable:
sns.set_palette("Set2")

In [ ]:
# 우선적으로 data provider즉 data가 2가지 정도 있는데 이는 몇개의 이미지가 있는지 분석해보자.
sns.countplot(x="data_provider", data=train)
# 그림의 결과 차이는 별로 없음으로 괜찮음.

In [ ]:
# 계속하여 gt의 label은 균등하게 분포가 되어있는지 보자.

fig, axs= plt.subplots(1,2,figsize=(15,6))
sns.countplot(y="gleason_score", data=train, ax=axs[0])
axs[0].set_title('gleason_score')
sns.countplot(y="gleason_score", hue="data_provider", data=train, ax=axs[1])
axs[1].set_title('gleason_score by data provider')


In [ ]:
fig, axs= plt.subplots(1,2,figsize=(10,6))
sns.countplot(x="isup_grade", data=train, ax=axs[0])
axs[0].set_title('isup grade')
sns.countplot(x="isup_grade", hue="data_provider", data=train, ax=axs[1])
axs[1].set_title('isup grade by data provider')


In [ ]:
## group마다 distribution을 살펴보자
data = [["Gleason Score", "ISUP Grade"],
        ["0+0", "0"], ["negative", "0"],
        ["3+3", "1"], ["3+4", "2"], ["4+3", "3"],
        ["4+4", "4"], ["3+5", "4"], ["5+3", "4"],
        ["4+5", "5"], ["5+4", "5"], ["5+5", "5"],
        ]## show sample images

tmp = train.groupby('data_provider')['gleason_score'].value_counts()
df = pd.DataFrame(data={'Exams': tmp.values}, index=tmp.index).reset_index()

In [ ]:
fig, axs= plt.subplots(1,2,figsize=(15,6))
sns.barplot(x = 'data_provider', y='Exams',hue='gleason_score',data=df, ax=axs[0])
sns.countplot(y="gleason_score", data=train, ax=axs[1])

#### result
위와 같이 살펴본결과 `karolinska`의 경우는 `negative, low score`일때 데이터가 많았으며   
`radboud`의 경우에서는 `high score`이 데이터가 일반적으로 많았다.  
또한 최종적으로 전체 데이터에서는 `3+3`의 데이터가 많았고 `3+5`의 데이터는 작았는데 이를 통해서 fold의 나눔에 도움을 줄것이다.

# 3. Load image and Anylsis
* 데이터를 분포를 파악했으니 이제 smaple이미지를 읽어보고 이를 어떻게 처리할지에 대해서 살펴보자.
1. data provider 마다 sample image를 읽어보기
2. Gleason score마다 data읽어보기
3. mask annoation과 함께 읽어보기
4. mask와 함께 데이터 tiling을 해보기

In [ ]:
def multi_plot(df, region=(0,0), level=-1, crop_size=(256,256)):
    f, ax = plt.subplots(3,3, figsize=(15,15))
    for i, row in enumerate(df.itertuples()):
        image = openslide.OpenSlide(row.img_path)
        slevel = image.level_count -1 if level == -1 else level
        patch = image.read_region(region, slevel, crop_size)
        ax[i//3, i%3].imshow(patch)
        image.close()
        ax[i//3, i%3].axis('off')

        image_id = row.image_id
        data_provider = row.data_provider
        isup_grade = row.isup_grade
        gleason_score = row.gleason_score
        ax[i//3, i%3].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score}")

    plt.show()

In [ ]:
sample_df = train.sample(9)
multi_plot(sample_df)

In [ ]:
id_list = [
    '037504061b9fba71ef6e24c48c6df44d',
    '035b1edd3d1aeeffc77ce5d248a01a53',
    '059cbf902c5e42972587c8d17d49efed',
    '06a0cbd8fd6320ef1aa6f19342af2e68',
    '06eda4a6faca84e84a781fee2d5f47e1',
    '0a4b7a7499ed55c71033cefb0765e93d',
    '0838c82917cd9af681df249264d2769c',
    '046b35ae95374bfb48cdca8d7c83233f',
    '074c3e01525681a275a42282cd21cbde',]

slect_df = train[train['image_id'].isin(id_list)]

In [ ]:
multi_plot(slect_df, (1780,1950), 0, (256,256))

In [ ]:
import matplotlib
def mask_multi_plot(df, region=(0,0), level=-1, crop_size=(256,256)):
    f, ax = plt.subplots(3,3, figsize=(15,15))
    for i, row in enumerate(df.itertuples()):
        image = openslide.OpenSlide(row.mask_path)
        slevel = image.level_count -1 if level == -1 else level
        patch = image.read_region(region, slevel, crop_size)
        cmap = matplotlib.colors.ListedColormap(['black', 'gray', 'green', 'yellow', 'orange', 'red'])
        ax[i//3, i%3].imshow(np.asarray(patch)[:,:,0], cmap=cmap, interpolation='nearest', vmin=0, vmax=5)
        image.close()
        ax[i//3, i%3].axis('off')

        image_id = row.image_id
        data_provider = row.data_provider
        isup_grade = row.isup_grade
        gleason_score = row.gleason_score
        ax[i//3, i%3].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score}")

    plt.show()

In [ ]:
mask_multi_plot(sample_df)

In [ ]:
mask_multi_plot(slect_df, (1780,1950), 0, (256,256))

In [ ]:
import PIL
#https://www.kaggle.com/code/spidyweb/prostate-cancer-panda-insight-eda## reference :
def override_image(df, region=(0,0), level=-1, crop_size=(256,256), center='radboud', alpha=0.8, max_size=(800, 800)):
    f, ax = plt.subplots(3,3, figsize=(15,15))
    for i, row in enumerate(df.itertuples()):

        slide = openslide.OpenSlide(row.img_path)
        mask = openslide.OpenSlide(row.mask_path)
        slevel = slide.level_count -1 if level == -1 else level
        slide_data = slide.read_region(region, slevel, crop_size)
        mask_data = mask.read_region(region, slevel, crop_size)
        mask_data = mask_data.split()[0]


        # Create alpha mask
        alpha_int = int(round(255*alpha))
        if center == 'radboud':
            alpha_content = np.less(mask_data.split()[0], 2).astype('uint8') * alpha_int + (255 - alpha_int)
        elif center == 'karolinska':
            alpha_content = np.less(mask_data.split()[0], 1).astype('uint8') * alpha_int + (255 - alpha_int)

        alpha_content = PIL.Image.fromarray(alpha_content)
        preview_palette = np.zeros(shape=768, dtype=int)

        if center == 'radboud':
            # Mapping: {0: background, 1: stroma, 2: benign epithelium, 3: Gleason 3, 4: Gleason 4, 5: Gleason 5}
            preview_palette[0:18] = (np.array([0, 0, 0, 0.5, 0.5, 0.5, 0, 1, 0, 1, 1, 0.7, 1, 0.5, 0, 1, 0, 0]) * 255).astype(int)
        elif center == 'karolinska':
            # Mapping: {0: background, 1: benign, 2: cancer}
            preview_palette[0:9] = (np.array([0, 0, 0, 0, 1, 0, 1, 0, 0]) * 255).astype(int)

        mask_data.putpalette(data=preview_palette.tolist())
        mask_rgb = mask_data.convert(mode='RGB')
        overlayed_image = PIL.Image.composite(image1=slide_data, image2=mask_rgb, mask=alpha_content)
        overlayed_image.thumbnail(size=max_size, resample=0)


        ax[i//3, i%3].imshow(overlayed_image)
        slide.close()
        mask.close()
        ax[i//3, i%3].axis('off')

        image_id = row.image_id
        data_provider = row.data_provider
        isup_grade = row.isup_grade
        gleason_score = row.gleason_score
        ax[i//3, i%3].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score}")


In [ ]:
override_image(sample_df)

In [ ]:
override_image(slect_df, (1780,1950), 0, (256,256))

In [ ]:
def print_slide_details(slide, show_thumbnail=True, max_size=(600,400)):
    """Print some basic information about a slide"""
    # Generate a small image thumbnail
    if show_thumbnail:
        display(slide.get_thumbnail(size=max_size))

    # Here we compute the "pixel spacing": the physical size of a pixel in the image.
    # OpenSlide gives the resolution in centimeters so we convert this to microns.
    spacing = 1 / (float(slide.properties['tiff.XResolution']) / 10000)

    print(f"File id: {slide}")
    print(f"Dimensions: {slide.dimensions}")
    print(f"Microns per pixel / pixel spacing: {spacing:.3f}")
    print(f"Number of levels in the image: {slide.level_count}")
    print(f"Downsample factor per level: {slide.level_downsamples}")
    print(f"Dimensions of levels: {slide.level_dimensions}")

In [ ]:
for row in sample_df.itertuples():
    biopsy = openslide.OpenSlide(row.img_path)
    print_slide_details(biopsy)
    biopsy.close()
    break

## Get all meta data

In [ ]:
from tqdm import tqdm
meta_dict = {'width':[], 'height':[], 'spacing':[], 'level_count':[]}
for row in tqdm(train.itertuples(), total=len(train)):
    slide = openslide.OpenSlide(row.img_path)

    spacing = 1 / (float(slide.properties['tiff.XResolution']) / 10000)

    meta_dict['width'].append(slide.dimensions[0])
    meta_dict['height'].append(slide.dimensions[1])
    meta_dict['spacing'].append(spacing)
    meta_dict['level_count'].append(slide.level_count)


In [ ]:
train_df = pd.concat([train, pd.DataFrame(meta_dict)], axis=1)

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = sns.scatterplot(x='width', y='height', data=train_df, alpha=0.3)
plt.title("height(y) width(x) scatter plot")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = sns.scatterplot(x='width', y='height', hue='isup_grade', data=train_df, alpha=0.6)
plt.title("height(y) width(x) scatter plot with target")
plt.show()

## Make tile

In [ ]:

def tile(img, mask):
    result = []
    shape = img.shape
    pad0,pad1 = (cfg.sz - shape[0]%cfg.sz)%cfg.sz, (cfg.sz - shape[1]%cfg.sz)%cfg.sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    mask = np.pad(mask,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=0)
    img = img.reshape(img.shape[0]//cfg.sz,cfg.sz,img.shape[1]//cfg.sz,cfg.sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,cfg.sz,cfg.sz,3)
    mask = mask.reshape(mask.shape[0]//cfg.sz,cfg.sz,mask.shape[1]//cfg.sz,cfg.sz,3)
    mask = mask.transpose(0,2,1,3,4).reshape(-1,cfg.sz,cfg.sz,3)
    if len(img) < cfg.N:
        mask = np.pad(mask,[[0,cfg.N-len(img)],[0,0],[0,0],[0,0]],constant_values=0)
        img = np.pad(img,[[0,cfg.N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:cfg.N]
    img = img[idxs]
    mask = mask[idxs]
    for i in range(len(img)):
        result.append({'img':img[i], 'mask':mask[i], 'idx':i})
    return result


In [ ]:
import zipfile
import skimage
def save_image(row):
    img = skimage.io.MultiImage(str(row.img_path))[-1]
    mask = skimage.io.MultiImage(str(row.mask_path))[-1]
    tiles = tile(img,mask)
    for t in tiles:
        img,mask,idx = t['img'],t['mask'],t['idx']
        x_tot.append((img/255.0).reshape(-1,3).mean(0))
        x2_tot.append(((img/255.0)**2).reshape(-1,3).mean(0))
        #if read with PIL RGB turns into BGR
        img = cv2.imencode('.png',cv2.cvtColor(img, cv2.COLOR_RGB2BGR))[1]
        img_out.writestr(f'{row.image_id}_{idx}.png', img)
        mask = cv2.imencode('.png',mask[:,:,0])[1]
        mask_out.writestr(f'{row.image_id}_{idx}.png', mask)


In [ ]:
from joblib import Parallel, delayed
faster = False
x_tot,x2_tot = [],[]
with zipfile.ZipFile(cfg.train_zip_path, 'w') as img_out,\
 zipfile.ZipFile(cfg.mask_zip_path, 'w') as mask_out:
    pbar = tqdm(train_df.itertuples(), total=len(train_df))
    [save_image(row) for row in pbar]

In [ ]:
#image stats
img_avr =  np.array(x_tot).mean(0)
img_std =  np.sqrt(np.array(x2_tot).mean(0) - img_avr**2)
print('mean:',img_avr, ', std:', np.sqrt(img_std))